In [ ]:
# execute cpm2cube.x while error message
# check output if sum > 1? -> give warning

In [3]:
import subprocess
import os
import shutil

import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')
import utils_qm as uqm

In [6]:
for k in run_info.keys():
    if not run_info[k] == 'reasonable density':
        print(k)

In [7]:
run_info()

{'/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/bond_stretch/butanol/dist_0.0/ve_13': 'reasonable density',
 '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/bond_stretch/butanol/dist_0.0/ve_19': 'reasonable density',
 '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/bond_stretch/butanol/dist_0.0/ve_26': 'reasonable density',
 '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/bond_stretch/butanol/dist_0.0/ve_32': 'reasonable density',
 '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/bond_stretch/butanol/dist_-0.125/ve_13': 'reasonable density',
 '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/bond_stretch/butanol/dist_-0.125/ve_19': 'reasonable density',
 '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/bond_stretch/butanol/dist_-0.125/ve_26': 'reasonable density',
 '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geom

In [5]:
uqm.load_obj('/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/dihedrals/butanol/cube_info')

{'/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/dihedrals/butanol/dihe_0/ve_32': 'reasonable density',
 '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/dihedrals/butanol/dihe_120/ve_32': 'reasonable density',
 '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/dihedrals/butanol/dihe_150/ve_32': 'reasonable density',
 '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/dihedrals/butanol/dihe_180/ve_32': 'reasonable density',
 '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/dihedrals/butanol/dihe_30/ve_32': 'reasonable density',
 '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/dihedrals/butanol/dihe_60/ve_32': 'reasonable density',
 '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/dihedrals/butanol/dihe_90/ve_32': 'reasonable density'}